# Problem Statement:
It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
                                'londON_StockhOlm',
                                'Budapest_PaRis', 'Brussels_londOn'],

'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],

'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],

'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',

'12. Air France', '"Swiss Air"']})

In [148]:
# Import Libraries
import pandas as pd
import numpy as np

In [169]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'],
                   'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                   'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']
                   })
df.head()

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [150]:
df.isnull().sum()

From_To         0
FlightNumber    2
RecentDelays    0
Airline         0
dtype: int64

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   From_To       5 non-null      object 
 1   FlightNumber  3 non-null      float64
 2   RecentDelays  5 non-null      object 
 3   Airline       5 non-null      object 
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes


In [170]:
df['FlightNumber'].interpolate(method = 'linear', axis = 0, inplace = True)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [172]:
df_temp = df.copy()
df_temp[['From', 'To']]  = df['From_To'].str.split('_', expand = True)
df_temp.drop(['From_To'], axis = 1, inplace = True)
df_temp


,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),LoNDon,paris
1,10055.0,[],<Air France> (12),MAdrid,miLAN
2,10065.0,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,10075.0,[13],12. Air France,Budapest,PaRis
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [173]:
df['From_To'] = df['From_To'].str.capitalize()
df

,From_To,FlightNumber,RecentDelays,Airline
0,London_paris,10045.0,"[23, 47]",KLM(!)
1,Madrid_milan,10055.0,[],<Air France> (12)
2,London_stockholm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_paris,10075.0,[13],12. Air France
4,Brussels_london,10085.0,"[67, 32]","""Swiss Air"""


In [174]:
df_temp['From'] = df_temp['From'].str.capitalize()
df_temp['To'] = df_temp['To'].str.capitalize()
df_temp

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,10055.0,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075.0,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [175]:
df.drop(['From_To'], axis = 1, inplace = True)

In [196]:
df = pd.concat([df_temp.iloc[:,[3,4]], df], axis = 1) 

In [197]:
df

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045.0,"[23, 47]",KLM(!)
1,Madrid,Milan,10055.0,[],<Air France> (12)
2,London,Stockholm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075.0,[13],12. Air France
4,Brussels,London,10085.0,"[67, 32]","""Swiss Air"""


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each

second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [215]:
df['RecentDelays'].values.tolist()

[[23, 47], [], [24, 43, 87], [13], [67, 32]]

In [227]:
d1 = df.copy()
d2 = (pd.DataFrame(d1['RecentDelays'].values.tolist()).rename(columns = lambda x: 'Delay_{}'.format(x+1)))

df = d1.join(d2)

In [229]:
df.drop(['RecentDelays'], axis = 1, inplace = True)
df

,From,To,FlightNumber,Airline,Delay_1,Delay_2,Delay_3
0,London,Paris,10045.0,KLM(!),23.0,47.0,NaN
1,Madrid,Milan,10055.0,<Air France> (12),NaN,NaN,NaN
2,London,Stockholm,10065.0,(British Airways. ),24.0,43.0,87.0
3,Budapest,Paris,10075.0,12. Air France,13.0,NaN,NaN
4,Brussels,London,10085.0,"""Swiss Air""",67.0,32.0,NaN
